In [310]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [312]:
df = pd.read_csv('./Impact_of_Remote_Work_on_Mental_Health.csv')

In [314]:
df.head()

,Employee_ID,Age,Gender,Job_Role,Industry,Years_of_Experience,Work_Location,Hours_Worked_Per_Week,Number_of_Virtual_Meetings,Work_Life_Balance_Rating,Stress_Level,Mental_Health_Condition,Access_to_Mental_Health_Resources,Productivity_Change,Social_Isolation_Rating,Satisfaction_with_Remote_Work,Company_Support_for_Remote_Work,Physical_Activity,Sleep_Quality,Region
0,EMP0001,32,Non-binary,HR,Healthcare,13,Hybrid,47,7,2,Medium,Depression,No,Decrease,1,Unsatisfied,1,Weekly,Good,Europe
1,EMP0002,40,Female,Data Scientist,IT,3,Remote,52,4,1,Medium,Anxiety,No,Increase,3,Satisfied,2,Weekly,Good,Asia
2,EMP0003,59,Non-binary,Software Engineer,Education,22,Hybrid,46,11,5,Medium,Anxiety,No,No Change,4,Unsatisfied,5,NaN,Poor,North America
3,EMP0004,27,Male,Software Engineer,Finance,20,Onsite,32,8,4,High,Depression,Yes,Increase,3,Unsatisfied,3,NaN,Poor,Europe
4,EMP0005,49,Male,Sales,Consulting,32,Onsite,35,12,2,High,NaN,Yes,Decrease,3,Unsatisfied,3,Weekly,Average,North America


In [316]:
df.shape

(5000, 20)

In [318]:
df.isnull().mean()

Employee_ID                          0.0000
Age                                  0.0000
Gender                               0.0000
Job_Role                             0.0000
Industry                             0.0000
Years_of_Experience                  0.0000
Work_Location                        0.0000
Hours_Worked_Per_Week                0.0000
Number_of_Virtual_Meetings           0.0000
Work_Life_Balance_Rating             0.0000
Stress_Level                         0.0000
Mental_Health_Condition              0.2392
Access_to_Mental_Health_Resources    0.0000
Productivity_Change                  0.0000
Social_Isolation_Rating              0.0000
Satisfaction_with_Remote_Work        0.0000
Company_Support_for_Remote_Work      0.0000
Physical_Activity                    0.3258
Sleep_Quality                        0.0000
Region                               0.0000
dtype: float64

In [320]:
df.dtypes

Employee_ID                          object
Age                                   int64
Gender                               object
Job_Role                             object
Industry                             object
Years_of_Experience                   int64
Work_Location                        object
Hours_Worked_Per_Week                 int64
Number_of_Virtual_Meetings            int64
Work_Life_Balance_Rating              int64
Stress_Level                         object
Mental_Health_Condition              object
Access_to_Mental_Health_Resources    object
Productivity_Change                  object
Social_Isolation_Rating               int64
Satisfaction_with_Remote_Work        object
Company_Support_for_Remote_Work       int64
Physical_Activity                    object
Sleep_Quality                        object
Region                               object
dtype: object

In [322]:
df['Satisfaction_with_Remote_Work'].value_counts()

Satisfaction_with_Remote_Work
Unsatisfied    1677
Satisfied      1675
Neutral        1648
Name: count, dtype: int64

In [324]:
from sklearn.preprocessing import LabelEncoder

In [326]:
le = LabelEncoder()

In [328]:
df['target']=le.fit_transform(df['Satisfaction_with_Remote_Work'])

In [330]:
df = df.drop(columns=['Employee_ID','Satisfaction_with_Remote_Work'],axis=1)

In [332]:
x_cat = df.select_dtypes(include='object')
x_num = df.select_dtypes(exclude='object')

In [334]:
x = df.drop('target',axis=1)
y = df['target']

In [336]:
x_num = x_num.drop('target',axis=1)

In [340]:
x_cat.head()

,Gender,Job_Role,Industry,Work_Location,Stress_Level,Mental_Health_Condition,Access_to_Mental_Health_Resources,Productivity_Change,Physical_Activity,Sleep_Quality,Region
0,Non-binary,HR,Healthcare,Hybrid,Medium,Depression,No,Decrease,Weekly,Good,Europe
1,Female,Data Scientist,IT,Remote,Medium,Anxiety,No,Increase,Weekly,Good,Asia
2,Non-binary,Software Engineer,Education,Hybrid,Medium,Anxiety,No,No Change,NaN,Poor,North America
3,Male,Software Engineer,Finance,Onsite,High,Depression,Yes,Increase,NaN,Poor,Europe
4,Male,Sales,Consulting,Onsite,High,NaN,Yes,Decrease,Weekly,Average,North America


In [342]:
x_num.head()

,Age,Years_of_Experience,Hours_Worked_Per_Week,Number_of_Virtual_Meetings,Work_Life_Balance_Rating,Social_Isolation_Rating,Company_Support_for_Remote_Work
0,32,13,47,7,2,1,1
1,40,3,52,4,1,3,2
2,59,22,46,11,5,4,5
3,27,20,32,8,4,3,3
4,49,32,35,12,2,3,3


In [344]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [354]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('feature_scaling', MinMaxScaler())
            ]), x_num.columns),
        ('cat', Pipeline(steps=[
                ('imputer', SimpleImputer(missing_values=np.nan,strategy='most_frequent')),
                ('encoding', OneHotEncoder())
            ]),x_cat.columns)
    ]
)

In [356]:
pipeline= Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('logistic_regression', LogisticRegression())
])

In [358]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [360]:
pipeline.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('feature_scaling',
                                                                   MinMaxScaler())]),
                                                  Index(['Age', 'Years_of_Experience', 'Hours_Worked_Per_Week',
       'Number_of_Virtual_Meetings', 'Work_Life_Balance_Rating',
       'Social_Isolation_Rating', 'Company_Support_for_Remote_Work'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoding',
                                                                   OneHotEncoder())]),
                                                  Index(['Gender', 'Job_Role', 'Industry', 'Work_Location', 'Stress_Level',
       'Mental_Health_Condition', 'Access_to_Mental_Health_Resources',
       'Productivity_Change', 'Physical_Activity', 'Sleep_Quality', 'Region'],
      dtype='object'))])),
                ('logistic_regression', LogisticRegression())])

In [362]:
y_pred = pipeline.predict(x_test)

In [364]:
metrics.accuracy_score(y_test,y_pred)

0.338

In [244]:
x_train.shape

(4000, 18)

In [248]:
y_train

4227    0
4676    2
800     1
3671    1
4193    2
       ..
4426    2
466     0
3092    0
3772    1
860     2
Name: target, Length: 4000, dtype: int32